The example code below shows how to instantiate a HeatmapPlotter object from the plotter module and use it to perform real-time heatmap plotting of the delta ADC image.

By default, the plot is generated directly below the code cell and the color scale used is 'Viridis'. These settings can be adjusted by passing in parameters when instantiating the HeatmapPlotter object. For example, *plotter = HeatmapPlotter(mode='jupyterlab')* generates the plot in a separate tab in JupyterLab instead of directly below the code cell. The available parameters are listed below.

**mode** (default = 'inline')<br/>
&emsp;'inline' - Generate plot directly below code cell<br/>
&emsp;'jupyterlab' - Generate plot in separate tab in JupyterLab<br/>
&emsp;'external' - Generate hyperlink for displaying plot in separate browser tab

**colorscale** (default = 'Viridis')<br/>
&emsp;A full listing of the supported color scales can be found at https://plotly.com/python/builtin-colorscales/.

**plotscale** (default = 20)<br/>
&emsp;This is a pixel-based multiplier applied to the numbers of rows and columns of the image to determine the size of the plot.

To terminate the plotting session, go to the Kernel main menu and do Interrupt Kernel. This triggers the KeyboardInterrupt exception shown at the bottom of the example code.
<br/><br/>
***
**Note**: The plotter module makes use of JupyterDash, which is served from the DSDK through port 8050. This means that in order for plotting to work, the "adb forward tcp:8050 tcp:8050" command needs to be issued first from the host computer this browser is running on.
***

In [ ]:
import sys
import time
import import_ipynb
from plotter import HeatmapPlotter
from touchcomm import TouchComm

fps = 60
period = 1/fps

tcm = TouchComm.make('report_streamer', streaming=False)
if (tcm):
    tcm.reset()
    tcm.disableReport(17)
    tcm.enableReport(18)
else:
    sys.exit(1)

try:
    plotter = HeatmapPlotter()
except Exception as e:
    tcm.close()
    sys.exit(e)

try:
    plotter.run_plot()
    while True:
        report = tcm.getReport()
        if ('delta' in report or 'raw' in report):
            image = report[1]['image']
            plotter.update_plot(image)
        time.sleep(period)
except KeyboardInterrupt:
    plotter.stop_plot()
finally:
    tcm.close()
